In [1]:
import pandas as pd
import numpy as np
import json
import re
from ast import literal_eval
from fuzzywuzzy import fuzz

In [2]:
data_path="../data/"
output_path="../outputs/"

In [3]:
tpredicted = [json.loads(line) for line in open('../../../test_outputs/sg_papers_scirex_predictions_test_final_2.jsonl')]
tdf_predicted = pd.DataFrame(tpredicted)
salient_test = [json.loads(line) for line in open('../../../test_outputs/salient_mentions_predictions_sg_papers_scirex_test_final_2.jsonl')]
df_salient_test = pd.DataFrame(salient_test)
sclusters = [json.loads(line) for line in open('../../../test_outputs/sg_papers_scirex_cluster_predictions_test_final_2.jsonl')]
tdf_sclusters = pd.DataFrame(sclusters)

df_salient_test=tdf_predicted.merge(df_salient_test,on='doc_id',how='left')
df_salient_test=df_salient_test.merge(tdf_sclusters,on='doc_id',how='left')



In [4]:
df_salient_test.shape

(5088, 8)

In [5]:
df_salient=df_salient_test

In [6]:
df_salient['Method']=[list() for _ in range(df_salient.shape[0])]
df_salient['Task']=[list() for _ in range(df_salient.shape[0])]
df_salient['Metric']=[list() for _ in range(df_salient.shape[0])]
df_salient['Dataset']=[list() for _ in range(df_salient.shape[0])]
df_salient['Material']=[list() for _ in range(df_salient.shape[0])]

In [7]:
df_salient=df_salient.assign(clusters=np.where(df_salient.clusters.isna(),None,df_salient.clusters))
df_salient=df_salient.assign(saliency=np.where(df_salient.saliency.isna(),None,df_salient.saliency))

In [8]:
df_salient.loc[(df_salient.saliency.isna()) & (df_salient.clusters.isna())]

,words,sections,sentences,ner,doc_id,saliency,spans,clusters,Method,Task,Metric,Dataset,Material
94,"[Assigning, Verbs, to, Semantic, Classes, via]","[[0, 6]]","[[0, 6]]",[],korhonen-2002-assigning,None,NaN,None,[],[],[],[],[]
229,"[Predicative, multi, -, word, expressions, in,...","[[0, 84]]","[[0, 24], [24, 43], [43, 64], [64, 84], [84, 84]]",[],fleischhauer-2020-predicative,None,NaN,None,[],[],[],[],[]
291,"[Resolving, Event, Noun, Phrases, to, Their, V...","[[0, 7]]","[[0, 7]]",[],chen-etal-2010-resolving,None,NaN,None,[],[],[],[],[]
302,"[Parallels, between, Linguistics, and, Biology...","[[0, 41]]","[[0, 19], [19, 41], [41, 41]]",[],chakraborti-tendulkar-2013-parallels,None,NaN,None,[],[],[],[],[]
311,"[Semantically, Analyzing, an, English, Subset,...","[[0, 8]]","[[0, 8]]",[],simmons-bennett-novak-1975-semantically,None,NaN,None,[],[],[],[],[]
352,"[On, the, Means, for, Clarification, in]","[[0, 6]]","[[0, 6]]",[],purver-etal-2001-means,None,NaN,None,[],[],[],[],[]
471,"[Parsing, English, Conjunctions, And, Comparat...","[[0, 12]]","[[0, 12]]",[],liu-soo-1989-parsing,None,NaN,None,[],[],[],[],[]
646,"[Using, prepositions, to, extend, a, verb]","[[0, 6]]","[[0, 6]]",[],kipper-etal-2004-using,None,NaN,None,[],[],[],[],[]
719,"[Doing, Dutch, Pronouns, Automatically, in, Op...","[[0, 6]]","[[0, 6]]",[],bouma-2003-dutch,None,NaN,None,[],[],[],[],[]
1155,"[Resolving, Pronouns, Robustly, :, Plumbing, t...","[[0, 8]]","[[0, 8]]",[],siddharthan-2003-resolving,None,NaN,None,[],[],[],[],[]


In [9]:
lsaliency=[]
for i,row in df_salient.iterrows():
    #print(row['ner'])
    ner={}
    score={}
    saliency={}
    doc_dict={}
    ner_list=[]
    
    doc_ner={}
    for n in row['ner']:
        word=str.lower(' '.join(row['words'][n[0]:n[1]]))
        if n[2] in doc_ner:
            doc_ner[n[2]].append({'words':[word],'value':0,'top_word':word})
        else:
            doc_ner[n[2]]=list([{'words':[word],'value':0,'top_word':word}])
    if (row['clusters']!=None):
        for s,n in zip(row['saliency'],row['ner']):
            ner[str(n[0])+"_"+str(n[1])]=n[2]
            score[str(n[0])+"_"+str(n[1])]=s[3]
            saliency[str(n[0])+"_"+str(n[1])]=s[2]
        for k in row['clusters'].keys():
            scores=[]
            saliency_flag=0
            ner_list=[]
            word_list=[]
            for entity in row['clusters'][k]:
                #print((' '.join(row['words'][entity[0]:entity[1]])).lower())
                #print(ner[str(entity[0])+"_"+str(entity[1])])
                #print(score[str(entity[0])+"_"+str(entity[1])])
                ner_list.append(ner[str(entity[0])+"_"+str(entity[1])])
                scores.append(score[str(entity[0])+"_"+str(entity[1])])
                word=(' '.join(row['words'][entity[0]:entity[1]])).lower()
                word=re.sub('[^0-9a-zA-Z ]+', '', word)
                word=re.sub(' +', ' ',word)
                word_list.append(word)
                if saliency[str(entity[0])+"_"+str(entity[1])]==1:
                    saliency_flag=1
            #print(word_list)
            type_ent=set(ner_list).pop()
            if (type_ent in doc_dict) and (type_ent in ['Method','Task','Metric','Dataset']):
                doc_dict[type_ent].append({'words':word_list,'value':np.mean(scores),'top_word':word_list[scores.index(max(scores))]})
            else:
                doc_dict[type_ent]=list([{'words':word_list,'value':np.mean(scores),'top_word':word_list[scores.index(max(scores))]}])
    #print(doc_ner)
    #print()
    #print(doc_dict)
    #print("#########")
    
    for key in doc_ner.keys():
        if key not in doc_dict.keys():
            doc_dict[key]=doc_ner[key]
    for key in doc_dict.keys():
        df_salient.at[i,key]=doc_dict[key]
        
        #print()

In [10]:
df_labels_task=df_salient.loc[:,['doc_id','Task','Method']].rename(columns={'doc_id':'ID'})

In [11]:
df_labels_task=df_labels_task.rename(columns={'Task':'task_scirex','Method':'method_scirex'})

In [12]:
df_labels_task.loc[(df_labels_task.method_scirex.apply(lambda x:len(x)!=0))].shape

(4647, 3)

In [13]:
df_labels_task.loc[(df_labels_task.method_scirex.apply(lambda x:len(x)==0)) & (df_labels_task.task_scirex.apply(lambda x:len(x)==0))]

,ID,task_scirex,method_scirex
33,ohno-etal-2006-syntactically,[],[]
94,korhonen-2002-assigning,[],[]
229,fleischhauer-2020-predicative,[],[]
291,chen-etal-2010-resolving,[],[]
302,chakraborti-tendulkar-2013-parallels,[],[]
311,simmons-bennett-novak-1975-semantically,[],[]
352,purver-etal-2001-means,[],[]
471,liu-soo-1989-parsing,[],[]
605,sloos-etal-2018-boarnsterhim,[],[]
612,grice-savino-1997-pitch,[],[]


In [14]:
df_labels_task.loc[(df_labels_task.method_scirex.apply(lambda x:len(x)!=0)) | (df_labels_task.task_scirex.apply(lambda x:len(x)!=0))]

,ID,task_scirex,method_scirex
0,miller-etal-2012-using,"[{'words': ['disambiguation', 'disambiguation'...","[{'words': ['knowledge based methods'], 'value..."
1,kim-etal-2019-unsupervised,"[{'words': ['parsing', 'constituency grammar i...",[{'words': ['unsupervised recurrent neural net...
2,iyer-etal-2021-veealign,"[{'words': ['ontology alignment', 'ontology al...",[{'words': ['multifaceted context representati...
3,patra-etal-2013-automatic,[{'words': ['automatic music mood classificati...,"[{'words': ['automatic methods'], 'value': 0, ..."
4,kazi-etal-2014-mitll,"[{'words': ['mt', 'mt', 'translating'], 'value...","[{'words': ['deep neural networks'], 'value': ..."
...,...,...,...
5083,owoputi-etal-2013-improved,"[{'words': ['part of speech tagging', 'part of...","[{'words': ['pos', 'tagging software'], 'value..."
5084,kang-etal-2020-neural,"[{'words': ['question answering', 'question an...",[{'words': ['reinforcement learning based fram...
5085,ligozat-2013-question,"[{'words': ['question classification', 'questi...",[]
5086,hozumi-etal-1993-integration,[{'words': ['morphological and syntactic analy...,"[{'words': ['lr', 'lr', 'lr'], 'value': 0.1034..."


In [15]:
df_labels_task.loc[(df_labels_task.method_scirex.apply(lambda x:len(x)!=0)) & (df_labels_task.task_scirex.apply(lambda x:len(x)!=0))]

,ID,task_scirex,method_scirex
0,miller-etal-2012-using,"[{'words': ['disambiguation', 'disambiguation'...","[{'words': ['knowledge based methods'], 'value..."
1,kim-etal-2019-unsupervised,"[{'words': ['parsing', 'constituency grammar i...",[{'words': ['unsupervised recurrent neural net...
2,iyer-etal-2021-veealign,"[{'words': ['ontology alignment', 'ontology al...",[{'words': ['multifaceted context representati...
3,patra-etal-2013-automatic,[{'words': ['automatic music mood classificati...,"[{'words': ['automatic methods'], 'value': 0, ..."
4,kazi-etal-2014-mitll,"[{'words': ['mt', 'mt', 'translating'], 'value...","[{'words': ['deep neural networks'], 'value': ..."
...,...,...,...
5082,narsale-2010-jhu,"[{'words': ['wmt 2010', 'wmt 2010 submission']...","[{'words': ['jhu system combination scheme', '..."
5083,owoputi-etal-2013-improved,"[{'words': ['part of speech tagging', 'part of...","[{'words': ['pos', 'tagging software'], 'value..."
5084,kang-etal-2020-neural,"[{'words': ['question answering', 'question an...",[{'words': ['reinforcement learning based fram...
5086,hozumi-etal-1993-integration,[{'words': ['morphological and syntactic analy...,"[{'words': ['lr', 'lr', 'lr'], 'value': 0.1034..."


In [16]:
df_labels_task.to_json(output_path+"sg_ie/test_scirex_tasks_methods_clusters_final_f.json")